In [ ]:
###downloaded and cleaned data. used sequential feature selection. built backtesting system to create good predictions, 
# evaluated error of predictions and improved that - chose ridge regression bc it runs very fast 

In [99]:
import os
import pandas as pd
import numpy as np
from pybaseball import batting_stats

In [100]:
START = 2002
END = 2024

In [101]:
batting = batting_stats(START, END, qual=200)

In [102]:
batting.to_csv("batting.csv")

In [103]:
batting = batting.groupby("IDfg", group_keys=False).filter(lambda x: x.shape[0] > 1) #IDfg = unique pid 
batting

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA,L-WAR
0,1109,2002,Barry Bonds,SFG,37,143,403,612,149,70,...,NaN,NaN,NaN,0,0.127,0.191,NaN,NaN,NaN,12.7
1,1109,2004,Barry Bonds,SFG,39,147,373,617,135,60,...,NaN,NaN,NaN,0,0.124,0.164,NaN,NaN,NaN,11.9
9,15640,2022,Aaron Judge,NYY,30,157,570,696,177,87,...,118.4,246.0,0.609,404,0.169,0.287,NaN,NaN,NaN,11.6
2,1109,2003,Barry Bonds,SFG,38,130,390,550,133,65,...,NaN,NaN,NaN,0,0.135,0.223,NaN,NaN,NaN,10.2
16,13611,2018,Mookie Betts,BOS,25,136,520,614,180,96,...,110.6,217.0,0.500,434,0.220,0.270,NaN,NaN,NaN,10.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7593,9272,2018,Chris Davis,BAL,32,128,470,522,79,51,...,111.8,113.0,0.401,282,0.174,0.316,NaN,NaN,NaN,-2.6
7037,45,2012,Rod Barajas,PIT,36,104,321,361,66,44,...,NaN,0.0,NaN,0,0.147,0.258,NaN,NaN,NaN,-2.6
7186,319,2011,Adam Dunn,CHW,31,122,415,496,66,39,...,NaN,0.0,NaN,0,0.169,0.295,NaN,NaN,NaN,-2.9
7535,620,2002,Neifi Perez,KCR,29,145,554,585,131,104,...,NaN,NaN,NaN,0,0.130,0.187,NaN,NaN,NaN,-2.9


In [104]:
def next_season(player): 
    player = player.sort_values("Season")
    player["Next_WAR"] = player["WAR"].shift(-1)
    return player
batting = batting.groupby("IDfg", group_keys=False).apply(next_season)


/var/folders/7p/tr_qrhjs4bn71769j321x5n40000gn/T/ipykernel_57403/3133076915.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  batting = batting.groupby("IDfg", group_keys=False).apply(next_season)


In [105]:
null_count = batting.isnull().sum()
null_count

IDfg           0
Season         0
Name           0
Team           0
Age            0
            ... 
xBA         7294
xSLG        7294
xwOBA       7294
L-WAR          0
Next_WAR    1262
Length: 321, dtype: int64

In [106]:
complete_cols = list(batting.columns[null_count == 0])
complete_cols

['IDfg',
 'Season',
 'Name',
 'Team',
 'Age',
 'G',
 'AB',
 'PA',
 'H',
 '1B',
 '2B',
 '3B',
 'HR',
 'R',
 'RBI',
 'BB',
 'IBB',
 'SO',
 'HBP',
 'SF',
 'SH',
 'GDP',
 'SB',
 'CS',
 'AVG',
 'GB',
 'FB',
 'LD',
 'IFFB',
 'Pitches',
 'Balls',
 'Strikes',
 'IFH',
 'BU',
 'BUH',
 'BB%',
 'K%',
 'BB/K',
 'OBP',
 'SLG',
 'OPS',
 'ISO',
 'BABIP',
 'GB/FB',
 'LD%',
 'GB%',
 'FB%',
 'IFFB%',
 'HR/FB',
 'IFH%',
 'BUH%',
 'wOBA',
 'wRAA',
 'wRC',
 'Bat',
 'Rep',
 'Pos',
 'RAR',
 'WAR',
 'Dol',
 'Spd',
 'wRC+',
 'WPA',
 '-WPA',
 '+WPA',
 'RE24',
 'REW',
 'pLI',
 'PH',
 'WPA/LI',
 'Clutch',
 'FB% (Pitch)',
 'FBv',
 'SL%',
 'SLv',
 'CB%',
 'CBv',
 'CH%',
 'CHv',
 'wFB',
 'wSL',
 'wCB',
 'wCH',
 'wFB/C',
 'wSL/C',
 'wCB/C',
 'wCH/C',
 'O-Swing%',
 'Z-Swing%',
 'Swing%',
 'O-Contact%',
 'Z-Contact%',
 'Contact%',
 'Zone%',
 'F-Strike%',
 'SwStr%',
 'BsR',
 'Def',
 'wSB',
 'UBR',
 'Age Rng',
 'Off',
 'Lg',
 'wGDP',
 'Pull%',
 'Cent%',
 'Oppo%',
 'Soft%',
 'Med%',
 'Hard%',
 'TTO%',
 'AVG+',
 'BB%+',
 'K

In [107]:
batting = batting[complete_cols + ["Next_WAR"]].copy()
batting

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,Events,CStr%,CSW%,L-WAR,Next_WAR
5973,1,2006,Alfredo Amezaga,FLA,28,132,334,378,87,72,...,107,113,143,109,63,0,0.188,0.256,1.1,2.0
5366,1,2007,Alfredo Amezaga,FLA,29,133,400,448,105,80,...,101,112,109,113,75,0,0.175,0.227,2.0,1.2
5630,1,2008,Alfredo Amezaga,FLA,30,125,311,337,82,61,...,101,101,123,111,64,0,0.178,0.244,1.2,NaN
1224,2,2002,Garret Anderson,ANA,30,158,638,678,195,107,...,91,80,65,97,129,0,0.137,0.232,3.7,5.1
900,2,2003,Garret Anderson,ANA,31,159,638,673,201,119,...,101,80,90,99,109,0,0.164,0.252,5.1,0.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3009,30116,2022,Seiya Suzuki,CHC,27,111,397,446,104,66,...,105,130,106,95,106,290,0.232,0.312,2.1,3.0
1463,30116,2023,Seiya Suzuki,CHC,28,138,515,583,147,90,...,109,92,90,93,115,392,0.204,0.283,3.2,0.9
2004,30116,2024,Seiya Suzuki,CHC,29,47,184,207,50,32,...,128,102,69,95,125,132,0.213,0.305,1.0,NaN
5549,31347,2023,Zach Neto,LAA,22,84,289,329,65,39,...,82,103,81,108,97,216,0.161,0.290,1.1,0.9


In [108]:
batting.dtypes
batting.dtypes[batting.dtypes == "object"]

Name       object
Team       object
Dol        object
Age Rng    object
dtype: object

In [109]:
del batting["Dol"]

In [110]:
del batting["Age Rng"]

In [111]:
batting["team_code"] = batting["Team"].astype("category").cat.codes
batting["team_code"]

5973    12
5366    12
5630    12
1224     1
900      1
        ..
3009     6
1463     6
2004     6
5549    15
4057    15
Name: team_code, Length: 7294, dtype: int8

In [129]:
batting_full = batting.copy()
batting = batting.dropna()

In [130]:
from sklearn.linear_model import Ridge
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import TimeSeriesSplit


rr = Ridge(alpha=1) #higher reduces overfitting, lower is closer to pure regular linear regression

split = TimeSeriesSplit(n_splits=3) #splits data into 3 parts in a time series aware way

sfs = SequentialFeatureSelector(rr, n_features_to_select=20, direction="forward", cv=split, n_jobs=4)

In [131]:
removed_columns = ["Next_WAR", "Name", "Team", "IDfg", "Season"] #removing metadata to avoid overfitting, colmn we want, and text data
selected_columns=batting.columns[~batting.columns.isin(removed_columns)]

In [132]:
from sklearn.preprocessing import MinMaxScaler #more aggressive scaler to avoid issues w negative ratios later

scaler = MinMaxScaler()
batting.loc[:, selected_columns] = scaler.fit_transform(batting[selected_columns])

/var/folders/7p/tr_qrhjs4bn71769j321x5n40000gn/T/ipykernel_57403/668747896.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.         0.         0.05882353 ... 0.05882353 0.         0.        ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  batting.loc[:, selected_columns] = scaler.fit_transform(batting[selected_columns])


In [133]:
batting.describe()

,IDfg,Season,Age,G,AB,PA,H,1B,2B,3B,...,Events,CStr%,CSW%,L-WAR,Next_WAR,team_code,player_season,war_corr,war_diff,war_season
count,4770.000000,4770.000000,4770.000000,4770.000000,4770.000000,4770.000000,4770.000000,4770.000000,4770.000000,4770.000000,...,4770.000000,4770.000000,4770.000000,4770.000000,4770.000000,4770.000000,4770.000000,4770.000000,4770.000000,4770.000000
mean,5834.972117,2012.448218,0.360805,0.684187,0.511067,0.516842,0.385826,0.302960,0.409565,0.100711,...,0.220090,0.495476,0.523659,0.349080,1.813040,0.473499,0.173733,0.530279,0.041513,0.350960
std,5547.892094,5.857927,0.144859,0.245361,0.235060,0.253705,0.179813,0.137659,0.167896,0.105632,...,0.300756,0.137499,0.134009,0.132279,1.987515,0.307696,0.163633,0.341154,0.025724,0.131482
min,1.000000,2003.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,-3.100000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1192.500000,2007.000000,0.240000,0.529915,0.325540,0.314879,0.241379,0.198113,0.271186,0.043478,...,0.000000,0.404255,0.435644,0.253333,0.400000,0.205882,0.058824,0.237989,0.028951,0.255311
50%,4082.000000,2012.000000,0.360000,0.752137,0.548561,0.557093,0.392241,0.301887,0.406780,0.086957,...,0.000000,0.493617,0.524752,0.326667,1.500000,0.470588,0.117647,0.536379,0.037836,0.331709
75%,9774.000000,2017.000000,0.440000,0.888889,0.708633,0.733564,0.525862,0.400943,0.525424,0.130435,...,0.487062,0.587234,0.613861,0.426667,2.900000,0.735294,0.294118,0.851726,0.049267,0.427210
max,30116.000000,2023.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,11.900000,1.000000,1.000000,1.000000,1.000000,1.000000


In [134]:
sfs.fit(batting[selected_columns], batting["Next_WAR"]) #fitting = picking the 20 predictors that give greatest accuracy with ridge regression modesl

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=Ridge(alpha=1), n_features_to_select=20,
                          n_jobs=4)

In [135]:
sfs.get_support() #True is columns we want to select
predictors = list(selected_columns[sfs.get_support()])

In [136]:
def backtest(data, model, predictors, start=5, step=1): #cross-validation - use all other rows to help make prediction for a given row:
    all_predictions = []

    years = sorted(data["Season"].unique())

    for i in range(start, len(years), step):
        current_year = years[i]

        train = data[data["Season"] < current_year]
        test = data[data["Season"] == current_year]

        model.fit(train[predictors], train["Next_WAR"])

        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        combined = pd.concat([test["Next_WAR"], preds], axis=1) #concatenating on axis 1 means each series should be treated as a separate column
        combined.columns = ["actual", "prediction"]

        all_predictions.append(combined)
    return pd.concat(all_predictions) #combines all dfs verticall into 1 long df


In [137]:
predictions = backtest(batting, rr, predictors)

In [138]:
predictions

,actual,prediction
3571,-1.1,0.817038
4645,0.7,0.232024
849,-0.1,3.073421
5550,1.5,1.305891
3432,1.1,0.890351
...,...,...
6452,1.9,1.843137
3674,-0.7,1.978726
3326,2.1,2.431677
3582,0.8,1.574725


In [139]:
from sklearn.metrics import mean_squared_error

mean_squared_error(predictions["actual"], predictions["prediction"])

2.6566816727449782

In [140]:
batting["Next_WAR"].describe()

count    4770.000000
mean        1.813040
std         1.987515
min        -3.100000
25%         0.400000
50%         1.500000
75%         2.900000
max        11.900000
Name: Next_WAR, dtype: float64

In [124]:
def player_history(df):
    df = df.sort_values("Season")

    df["player_season"] = range(0, df.shape[0])
    df["war_corr"] = list(df[["player_season", "WAR"]].expanding().corr().loc[(slice(None), "player_season"), "WAR"]) #gets correlation between previous seasons for all seasons
    df["war_corr"] = df["war_corr"].fillna(1)

    df["war_diff"] = df["WAR"] / df ["WAR"].shift(1) #takes WAR of current season adn divdes by prev season
    df["war_diff"].fillna(1)

    df["war_diff"] = df["war_diff"].replace(np.inf, 1)

    return df

batting = batting.groupby("IDfg", group_keys=False).apply(player_history)


/var/folders/7p/tr_qrhjs4bn71769j321x5n40000gn/T/ipykernel_57403/3080534936.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  batting = batting.groupby("IDfg", group_keys=False).apply(player_history)


In [125]:
def group_averages(df): #corrects for exceptions - e.g. lesser games bc of a lockout or something
    return df["WAR"]/df["WAR"].mean()

In [126]:
batting["war_season"] = batting.groupby("Season", group_keys=False).apply(group_averages)

/var/folders/7p/tr_qrhjs4bn71769j321x5n40000gn/T/ipykernel_57403/4191086496.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  batting["war_season"] = batting.groupby("Season", group_keys=False).apply(group_averages)


In [127]:
new_predictors = predictors + ["player_season", "war_corr", "war_season", "war_diff"]

In [141]:
predictions = backtest(batting, rr, new_predictors)

In [142]:
mean_squared_error(predictions["actual"], predictions["prediction"])

2.675446726887805

In [145]:
pd.Series(rr.coef_, index=new_predictors).sort_values() #large coeff is biggest w predictions

war_diff        -5.404145
Age             -2.983892
BABIP           -1.626373
PH              -1.076490
Soft%+          -1.035826
ISO+            -0.947813
SH              -0.855889
O-Swing%        -0.718032
Pull%           -0.552537
SO              -0.486266
war_corr        -0.316088
LD+%            -0.239392
Pos              0.214299
player_season    0.304105
Z-Swing%         0.372099
FB% (Pitch)      0.375363
IFH%             0.439675
SB               0.589280
O-Contact%       0.604105
Spd              0.893249
WAR              1.366534
IBB              1.976705
Hard%+           2.220624
war_season       5.561498
dtype: float64

In [147]:
diff = predictions["actual"] - predictions["prediction"]
diff

3571   -1.634361
4645    0.401969
849    -3.139705
5550    0.405020
3432    0.030475
          ...   
6452   -0.096879
3674   -2.389630
3326   -0.485827
3582   -0.863461
1463   -1.109492
Length: 3562, dtype: float64

In [150]:
merged = predictions.merge(batting, left_index=True, right_index=True)
merged["diff"] = (predictions["actual"] - predictions["prediction"]).abs()

In [151]:
merged[["IDfg", "Season", "Name", "WAR", "Next_WAR","diff"]].sort_values(["diff"])

,IDfg,Season,Name,WAR,Next_WAR,diff
401,9874,2016,DJ LeMahieu,0.540000,3.4,0.000534
6932,5223,2014,Cameron Maybin,0.226667,1.3,0.000935
7027,7476,2018,Alex Avila,0.260000,1.5,0.001687
3992,1702,2012,Reed Johnson,0.246667,-0.5,0.001914
1083,6364,2015,Danny Valencia,0.393333,1.7,0.002203
...,...,...,...,...,...,...
5491,1213,2009,Aubrey Huff,0.066667,5.7,6.118037
3463,5631,2010,Matt Kemp,0.206667,8.3,6.489658
4092,1875,2009,Josh Hamilton,0.293333,8.4,6.733587
2688,11579,2014,Bryce Harper,0.313333,9.3,7.469383
